In [39]:
import tweepy
import os
import pandas as pd
import yfinance as yf
import openai
import big_baller_moves
import json
import requests
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup

In [3]:
#Put your Bearer Token in the parenthesis below
client1 = tweepy.Client(bearer_token=big_baller_moves.bossman_tingz["twitter_api_key"])

# Authenticate
auth = tweepy.OAuth1UserHandler(big_baller_moves.bossman_tingz["consumer_key"], big_baller_moves.bossman_tingz["consumer_secret"], big_baller_moves.bossman_tingz["access_token"], 
                                big_baller_moves.bossman_tingz["access_token_secret"])

# Create API object
api = tweepy.API(auth)

# openai key
openai.api_key = big_baller_moves.bossman_tingz["openai_api_key"]

In [4]:
#enter username to get the user ID
username = "Heisenberg_100k"

try:
    user = client1.get_user(username=username)
    user_id = user_id = user.data.id
    print("User ID for", username, ":", user_id)
except Exception as e:
    print("Error:", e)

User ID for Heisenberg_100k : 1750349614786691072


In [5]:
# #get btc price (not working atm)

# # Choose Token and Time
# timestamp_open = pd.Timestamp('2024-05-02 05:09:00', tz='UTC')
# unix_timestamp = int(timestamp_open.timestamp())
# unix_timestamp_24 = unix_timestamp + (24*60*60)
# unix_timestamp_48 = unix_timestamp + (48*60*60)
# ckey = f"https://api.coingecko.com/api/v3/coins/bitcoin/history?date={unix_timestamp}&localization=false"
# url = f"https://api.kraken.com/0/public/OHLC?pair=XBTUSD&interval=1&since={unix_timestamp}"
# ckey24 = f"https://api.coingecko.com/api/v3/coins/bitcoin/history?date={unix_timestamp_24}&localization=false"
# ckey48 = f"https://api.coingecko.com/api/v3/coins/bitcoin/history?date={unix_timestamp_48}&localization=false"

# try:
#         # Convert timestamp to Unix timestamp (Kraken API requires timestamp in seconds)
#         unix_timestamp = int(timestamp.timestamp())

#         # Kraken API endpoint for OHLC data
#         url = f"https://api.kraken.com/0/public/OHLC?pair=XBTUSD&interval=1&since={unix_timestamp}"

# response = requests.get(url)
#         if response.status_code == 200:
#             data = response.json()
#             if 'result' in data and 'XBTUSD' in data['result'] and data['result']['XBTUSD']:
#                 # The response contains a list of lists, where each inner list represents OHLC data for a specific interval
#                 # We extract the closing price from the last list (most recent data point)
#                 bitcoin_price = float(data['result']['XBTUSD'][-1][4])
#                 return bitcoin_price
#             else:
#                 print("No data available for the specified timestamp")
#                 return None
#         else:
#             print(f"Failed to retrieve data from Kraken API. Status code: {response.status_code}")
#             return None
#     except Exception as e:
#         print(f"An error occurred: {e}")
#         return None


In [6]:
# unix_timestamp

In [7]:
#get all tweets
tweets = client1.get_users_tweets(
    id= user_id,
    max_results=100,  # Number of tweets to retrieve (adjust as needed)
    tweet_fields=['id', 'text', 'created_at', 'entities', 'attachments'],  # Fields you want to retrieve for each tweet
    media_fields=['preview_image_url', 'url'],
    exclude = ['retweets','replies'],
    expansions=['attachments.media_keys', 'author_id']
)


In [8]:
# Process retrieved tweets
# for tweet in tweets.data:
#     print(f"Tweet ID: {tweet.id}, Text: {tweet.text}, Created At: {tweet.created_at}")
df = pd.DataFrame(tweets.data)

Tweet ID: 1790421057742205068, Text: $PARA Closed

In 13.11 (yesterday)

Out 13.24

+1%
+$65 profit

Just trying to reduce long exposure heading into tomorrow. https://t.co/GpCKwDrfky, Created At: 2024-05-14 16:36:59+00:00
Tweet ID: 1790415982328263151, Text: $INTC Closed

In 30.53 (yesterday)

Out 30.91

+1.3%
+$152 profit

Looking for a market dip tomorrow. Then reenter at a lower price, hopefully. https://t.co/VUk3AXsCDY, Created At: 2024-05-14 16:16:49+00:00
Tweet ID: 1790077422697693598, Text: Question:

Sell 1 naked GME call here.

Worth it?

Dangerous?

Only 1 shouldn’t be too bad?, Created At: 2024-05-13 17:51:30+00:00
Tweet ID: 1790077076105498901, Text: $PARA Open

See how Sony plays the buyout process.

Small. https://t.co/CceV23s8gb, Created At: 2024-05-13 17:50:07+00:00
Tweet ID: 1790026595027005823, Text: Sold my $INTC calls too soon. But shares doing well.

Anyone hopped in the INTC trade?, Created At: 2024-05-13 14:29:31+00:00
Tweet ID: 1790014939173630180, Text: $VKTX 

In [9]:
#extract urls for tweets with images
def get_display_url(entities):
    if isinstance(entities, dict) and "urls" in entities:
        urls = entities["urls"]
        display_urls = [url.get("display_url") for url in urls]
        return display_urls
    else:
        return 0

# Apply the function to make the display_urls a column in our dataframe
df["image_url"] = df["entities"].apply(get_display_url)

selected_columns = ['id', 'text', 'created_at', 'image_url']

# Creating a new DataFrame with selected columns
heisenberg_tweets = df[selected_columns].copy()

In [10]:
heisenberg_tweets

,id,text,created_at,image_url
0,1790421057742205068,$PARA Closed\n\nIn 13.11 (yesterday)\n\nOut 13...,2024-05-14 16:36:59+00:00,[pic.twitter.com/GpCKwDrfky]
1,1790415982328263151,$INTC Closed\n\nIn 30.53 (yesterday)\n\nOut 30...,2024-05-14 16:16:49+00:00,[pic.twitter.com/VUk3AXsCDY]
2,1790077422697693598,Question:\n\nSell 1 naked GME call here.\n\nWo...,2024-05-13 17:51:30+00:00,0
3,1790077076105498901,$PARA Open\n\nSee how Sony plays the buyout pr...,2024-05-13 17:50:07+00:00,[pic.twitter.com/CceV23s8gb]
4,1790026595027005823,Sold my $INTC calls too soon. But shares doing...,2024-05-13 14:29:31+00:00,0
...,...,...,...,...
93,1783943747711148219,"$SPY Open\n\nRisk $1,750 to make $750.\n\nI’m ...",2024-04-26 19:38:28+00:00,[pic.twitter.com/OjXQoj4O34]
94,1783941606837784711,$SPY credit spread to expire OTM.\n\n❤️ if you...,2024-04-26 19:29:57+00:00,[x.com/Heisenberg_100…]
95,1783854096040292837,$SOXL Closed\n\nIn 32.38 (April 19)\n\nOut 37....,2024-04-26 13:42:13+00:00,[pic.twitter.com/fmM5DMrJan]
96,1783543215695753574,$META 440. \n\nDo it for daddy.,2024-04-25 17:06:53+00:00,0


In [11]:
#helper counting function to check display_urls (can ignore)

def count_display_url(entities):
    if isinstance(entities, dict) and "urls" in entities:
        urls = entities["urls"]
        display_urls = [url.get("display_url") for url in urls]
        print(display_urls)
        return sum(1 for url in display_urls if url is not None)
    else:
        return 0

# Apply the function to count display_url
count = df["entities"].apply(count_display_url).sum()

print("Number of rows with display_url:", count)

['pic.twitter.com/GpCKwDrfky']
['pic.twitter.com/VUk3AXsCDY']
['pic.twitter.com/CceV23s8gb']
['pic.twitter.com/2G1TFQ8iaV']
['pic.twitter.com/LeXt1fPshM', 'pic.twitter.com/LeXt1fPshM']
['pic.twitter.com/HRF5z5GmYK']
['x.com/i/web/status/1…']
['x.com/i/web/status/1…']
['pic.twitter.com/FJwJ8S1RdE']
['x.com/i/web/status/1…', 'x.com/Heisenberg_100…']
['pic.twitter.com/shmUd4IV7w']
['pic.twitter.com/1FrUXsjpbc']
['pic.twitter.com/FsZa8r2C56']
['pic.twitter.com/WNy2WINVwq']
['pic.twitter.com/wZl8ISwGxb']
['pic.twitter.com/iigSxnAU8S']
['pic.twitter.com/qsWN4xdihq']
['pic.twitter.com/rL3Y94Ex9E']
['x.com/i/web/status/1…']
['x.com/i/web/status/1…', 'x.com/ShiKitYip1/sta…']
['pic.twitter.com/aIjR48ncY0']
['pic.twitter.com/64syimBVcM']
['pic.twitter.com/bMkPWxmKx4']
['x.com/i/web/status/1…']
['pic.twitter.com/9Wo5JfC8mO']
['pic.twitter.com/pm160UzPGD', 'pic.twitter.com/pm160UzPGD', 'pic.twitter.com/pm160UzPGD']
['pic.twitter.com/xtQMTvDNEY']
['pic.twitter.com/CcLC9F7DSF']
['pic.twitter.com/Upro

In [12]:
df[df['attachments'].isnull()]['entities']

2     {'annotations': [{'start': 24, 'end': 26, 'pro...
4     {'cashtags': [{'start': 8, 'end': 13, 'tag': '...
7     {'cashtags': [{'start': 0, 'end': 5, 'tag': 'V...
8     {'cashtags': [{'start': 0, 'end': 4, 'tag': 'G...
10    {'urls': [{'start': 280, 'end': 303, 'url': 'h...
11    {'cashtags': [{'start': 32, 'end': 36, 'tag': ...
12    {'urls': [{'start': 207, 'end': 230, 'url': 'h...
14    {'urls': [{'start': 278, 'end': 301, 'url': 'h...
19    {'cashtags': [{'start': 0, 'end': 4, 'tag': 'D...
24    {'cashtags': [{'start': 6, 'end': 11, 'tag': '...
25    {'cashtags': [{'start': 26, 'end': 31, 'tag': ...
26    {'urls': [{'start': 281, 'end': 304, 'url': 'h...
27    {'annotations': [{'start': 51, 'end': 52, 'pro...
28                                                  NaN
29    {'urls': [{'start': 279, 'end': 302, 'url': 'h...
33    {'urls': [{'start': 282, 'end': 305, 'url': 'h...
35                                                  NaN
36    {'cashtags': [{'start': 18, 'end': 22, 'ta

In [13]:
ticker_symbol = 'AAPL'

# Fetch historical data
stock_data = yf.Ticker(ticker_symbol)

# Get historical market data
historical_data = stock_data.history(period="max")

print(historical_data)

                                 Open        High         Low       Close  \
Date                                                                        
1980-12-12 00:00:00-05:00    0.099058    0.099488    0.099058    0.099058   
1980-12-15 00:00:00-05:00    0.094320    0.094320    0.093890    0.093890   
1980-12-16 00:00:00-05:00    0.087429    0.087429    0.086999    0.086999   
1980-12-17 00:00:00-05:00    0.089152    0.089582    0.089152    0.089152   
1980-12-18 00:00:00-05:00    0.091737    0.092167    0.091737    0.091737   
...                               ...         ...         ...         ...   
2024-05-07 00:00:00-04:00  183.201504  184.649537  181.074400  182.152924   
2024-05-08 00:00:00-04:00  182.602329  182.822032  181.204216  182.492477   
2024-05-09 00:00:00-04:00  182.312720  184.409882  181.863333  184.320007   
2024-05-10 00:00:00-04:00  184.899994  185.089996  182.130005  183.050003   
2024-05-13 00:00:00-04:00  185.440002  187.100006  184.619995  186.279999   

In [14]:
client = openai
df1 = df.copy().head(5)
pd.set_option('display.max_colwidth', None)

In [15]:
# #for astronomer
# system_msg = "Tell me if the message is suggesting opening a long, short or N/A and explain why. Some messages might indicate that they are shorting in the short-term but the long-term view indicates a bullish sentiment. In this case, we should tag the tweet as a short position. If the tweet indicates anything around taking profit or being in a short position call it a short."

In [16]:
#heisenberg
system_msg = "Tell me if the message is suggesting opening a long, short or N/A and explain why. Some messages might indicate that they are shorting in the short-term but the long-term view indicates a bullish sentiment. In this case, we should tag the tweet as a short position. If the tweet indicates anything around taking profit or being in a short position call it a short."

In [17]:
def get_response(text) :
    response = client.chat.completions.create(model="gpt-3.5-turbo", 
                                              messages=[ {"role":"system","content":"Tell me if the message is suggesting opening a long, short or N/A in one word. Some messages might indicate that they are shorting in the short-term but the long-term view indicates a bullish sentiment. In this case, we should tag the tweet as a short position. If the tweet indicates anything around taking profit or being in a short position call it a short." }, 
                                                        {"role":"user","content": text}])
   
    return response.choices[0].message.content.strip()



In [18]:
def get_response_guided(text) :
    response = client.chat.completions.create(model="gpt-3.5-turbo", 
                                              messages=[ {"role":"system","content":"Your job is to tag tweets as either opening a long position or short position or N/A. Provide a reason explaining why the tweet is a long or short or n/a." }, 
                                                        {"role":"user","content": "Taking profit"},
                                                        {"role":"assistant","content": "Short, the poster is taking profit."}])
   
    return response.choices[0].message.content.strip()


In [19]:
def get_response_image(text) :
    if text == 0:
        return "No image available"
    if not isinstance(text, str):
        print(text)
        text = text[0]
    try:
        response = client.chat.completions.create(model="gpt-4o", messages=[
                            {
                              "role": "user",
                              "content": [
                                {"type": "text", "text": "What kind of stock purchase is this image describing? If it is an option play like a call or a put please specify."},
                                {
                                  "type": "image_url",
                                  "image_url": {
                                    "url": text,
                                  },
                                },
                              ],
                            }
                          ], max_tokens=300,)
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(e)
        pass

In [20]:
def buy_or_sell_GPT(text) :
    try:
        response = client.chat.completions.create(model="gpt-4o", 
                                                   messages=[
        {
          "role": "user",
          "content": [
            {"type": "text", "text": "What is this message and transcribed image data telling us? Please respond in format [Ticker (if any)] [Buy/Sell/Neither]"},
            {
              "type": "text",
              "text": text
            },
          ],
        }
      ],
      max_tokens=300,)
   
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(e)
        pass

In [21]:
# df1['Direction (Guided)'] = df1['text'].apply(get_response_guided)
# df1['Direction'] = df1['text'].apply(get_response)

In [22]:

heisenberg_tweets['Image'] = heisenberg_tweets['image_url'].apply(get_response_image)


['pic.twitter.com/GpCKwDrfky']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['pic.twitter.com/VUk3AXsCDY']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['pic.twitter.com/CceV23s8gb']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['pic.twitter.com/2G1TFQ8iaV']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['pic.twitter.com/LeXt1fPshM', 'pic.twitter.com/LeXt1fPshM']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['pic.twitter.com/HRF5z5GmYK']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_i

Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['pic.twitter.com/y4uMyzO6vS']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['pic.twitter.com/y1dM3zFGzt']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['x.com/i/web/status/1…', 'x.com/kirk_barnard/s…']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['pic.twitter.com/RbpVmOVfeU']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['x.com/i/web/status/1…']
Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}
['pic.twitter.com/OjXQoj4O34']
Error c

In [23]:

# selected = ['text','Direction','Direction (Guided)']
# df1[selected]

In [33]:
ht_mini = heisenberg_tweets

In [34]:
def initialize_webdriver():
    options = Options()
    options.add_argument("--headless")  # Run in background without opening a browser window
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    # Disable images, developer tools, pop-ups, extensions to speed up loading
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-extensions")  
    options.add_argument("--disable-popup-blocking") 
    options.add_argument("--disable-dev-tools")  
    options.add_argument('blink-settings=imagesEnabled=false')  

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver


def get_jpg_url(driver, url):
    try:
        if isinstance(url, list):  # Extract first element if URL is a list
            url = url[0] if url else None
        elif not url:
            return None
        if not url.startswith('http'):
            url = f'https://{url}'
        print(url)
        driver.get(url)
        # Wait for potential redirects and the page to stabilize
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'img')))
        time.sleep(1) # Short sleep to ensure images are loaded
        
        images = driver.find_elements(By.TAG_NAME, "img")
        for img in images:
            img_src = img.get_attribute('src')
            if 'media' in img_src:
                return img_src

    except NoSuchElementException as e:
        print(f"Element not found for URL {url}: {e}")
    except TimeoutException as e:
        print(f"Request timed out for URL {url}: {e}")
    except Exception as e:
        print(f"General error processing URL {url}: {e}")
    return None

driver = initialize_webdriver()

try:
    ht_mini.loc[:, 'jpg_url'] = ht_mini['image_url'].apply(lambda x: get_jpg_url(driver, x))

finally:
    driver.quit()

https://pic.twitter.com/GpCKwDrfky
https://pic.twitter.com/VUk3AXsCDY
https://pic.twitter.com/CceV23s8gb
https://pic.twitter.com/2G1TFQ8iaV
https://pic.twitter.com/LeXt1fPshM
https://pic.twitter.com/HRF5z5GmYK
https://x.com/i/web/status/1…
https://x.com/i/web/status/1…
https://pic.twitter.com/FJwJ8S1RdE
https://x.com/i/web/status/1…
https://pic.twitter.com/shmUd4IV7w
https://pic.twitter.com/1FrUXsjpbc
https://pic.twitter.com/FsZa8r2C56
https://pic.twitter.com/WNy2WINVwq
https://pic.twitter.com/wZl8ISwGxb
https://pic.twitter.com/iigSxnAU8S
https://pic.twitter.com/qsWN4xdihq
https://pic.twitter.com/rL3Y94Ex9E
https://x.com/i/web/status/1…
https://x.com/i/web/status/1…
https://pic.twitter.com/aIjR48ncY0
https://pic.twitter.com/64syimBVcM
https://pic.twitter.com/bMkPWxmKx4
https://x.com/i/web/status/1…
https://pic.twitter.com/9Wo5JfC8mO
https://pic.twitter.com/pm160UzPGD
https://pic.twitter.com/xtQMTvDNEY
https://pic.twitter.com/CcLC9F7DSF
https://pic.twitter.com/UproWGtGsH
https://pic.twi

In [36]:
ht_mini.loc[:,'image_response'] = ht_mini['jpg_url'].apply(lambda x: get_response_image(x) if x != None else None)
ht_mini['image_response'] = ht_mini['image_response'].apply(lambda x: x.replace('$', '\$') if x and pd.notna(x) else x)
ht_mini['text'] = ht_mini['text'].apply(lambda x: x.replace('$', '\$') if pd.notna(x) else x)
display(ht_mini)

Error code: 400 - {'error': {'message': 'Invalid image.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image'}}


,id,text,created_at,image_url,Image,jpg_url,image_response
0,1790421057742205068,\$PARA Closed\n\nIn 13.11 (yesterday)\n\nOut 13.24\n\n+1%\n+\$65 profit\n\nJust trying to reduce long exposure heading into tomorrow. https://t.co/GpCKwDrfky,2024-05-14 16:36:59+00:00,[pic.twitter.com/GpCKwDrfky],None,https://pbs.twimg.com/media/GNjadkgacAAB0Gy?format=jpg&name=small,"This image is describing a stock short sale of 500 shares of the stock ""PARA"" at a limit price of 13.24. It is not depicting an option play like a call or a put."
1,1790415982328263151,"\$INTC Closed\n\nIn 30.53 (yesterday)\n\nOut 30.91\n\n+1.3%\n+\$152 profit\n\nLooking for a market dip tomorrow. Then reenter at a lower price, hopefully. https://t.co/VUk3AXsCDY",2024-05-14 16:16:49+00:00,[pic.twitter.com/VUk3AXsCDY],None,https://pbs.twimg.com/media/GNjV1NsbwAAwYeB?format=jpg&name=small,"The image shows details of a stock sale for Intel Corporation (INTC). This is a straightforward stock trade and not an option play like a call or put. Here are the details:\n\n- Ticker: INTC\n- The action is a sell order.\n- Quantity sold: 400 shares in total.\n- Sale price: \$30.91 per share.\n \nThis image reflects a limit sell order executed at a price of \$30.91 per share, and it is not related to options trading such as calls or puts."
2,1790077422697693598,Question:\n\nSell 1 naked GME call here.\n\nWorth it?\n\nDangerous?\n\nOnly 1 shouldn’t be too bad?,2024-05-13 17:51:30+00:00,0,No image available,None,None
3,1790077076105498901,\$PARA Open\n\nSee how Sony plays the buyout process.\n\nSmall. https://t.co/CceV23s8gb,2024-05-13 17:50:07+00:00,[pic.twitter.com/CceV23s8gb],None,https://pbs.twimg.com/media/GNehnPvbMAAq-N7?format=jpg&name=small,"This image describes a stock purchase, not an option play. The details are as follows:\n\n- The stock ticker is ""PARA.""\n- The purchase involved buying 500 shares at a limit price of \$13.11.\n- The order was filled at a price of \$13.1099 per share.\n- The transaction took place on May 13, 2024, at 10:46 AM.\n\nThis indicates a straightforward stock purchase and not a call or put option."
4,1790026595027005823,Sold my \$INTC calls too soon. But shares doing well.\n\nAnyone hopped in the INTC trade?,2024-05-13 14:29:31+00:00,0,No image available,None,None
...,...,...,...,...,...,...,...
93,1783943747711148219,"\$SPY Open\n\nRisk \$1,750 to make \$750.\n\nI’m looking for some consolidation with a little downside bias to next week.\n\nThink SPY max upside is probably 512-513. Downside probably 500-505.\n\nDon’t matter just stay below 515. https://t.co/OjXQoj4O34",2024-04-26 19:38:28+00:00,[pic.twitter.com/OjXQoj4O34],None,https://pbs.twimg.com/media/GMHXYfCaIAAYxZj?format=jpg&name=small,"This image describes a vertical spread options trade, specifically a credit spread.\n\nHere are the details:\n- **Underlying Asset**: SPY (SPDR S&P 500 ETF)\n- **Type of Option**: Call options\n- **Expiration Date**: Weekly options expiring on May 3, 2024\n- **Strike Prices**: 515 and 516\n- **Trade Type**: Vertical spread (credit)\n- **Quantity**: 25 contracts for each leg of the spread\n\nIn this vertical spread:\n- **Sell (short) 25 call options with a 515 strike price** at 1.7\n- **Buy (long) 25 call options with a 516 strike price** at 1.4\n\nSince the trade is a credit spread, the net credit received is:\n- Sell price of \$1.7 - Buy price of \$1.4 = \$0.3 per contract\n\nThe average fill price for this entire order is -25 contracts at \$0.30, indicating that the trader received a premium of \$0.30 per contract for a total net credit for this trade."
94,1783941606837784711,\$SPY credit spread to expire OTM.\n\n❤️ if you followed. https://t.co/cxHfzEp0Xt,2024-04-26 19:29:57+00:00,[x.com/Heisenberg_100…],None,None,None
95,1783854096040292837,\$SOXL Closed\n\nIn 32.38 (April 19)\n\nOut 37.57\n\nGood enough https://t.co/fmM5DMrJan,2024-04-26 13:42:13+00:00,[pic.twitter.com/fmM5DMrJan],None,https://pbs.twimg.com/media/GMGF2GvbEAEYOQ_?format=jpg&name=small

In [37]:
ht_mini['full_response'] = ht_mini['text'].astype(str) + '  TRANSCRIBED IMAGE DATA: ' + ht_mini['image_response'].astype(str)
ht_mini['buy_or_sell'] = ht_mini['full_response'].apply(buy_or_sell_GPT)

In [38]:
display(ht_mini)

,id,text,created_at,image_url,Image,jpg_url,image_response,full_response,buy_or_sell
0,1790421057742205068,\$PARA Closed\n\nIn 13.11 (yesterday)\n\nOut 13.24\n\n+1%\n+\$65 profit\n\nJust trying to reduce long exposure heading into tomorrow. https://t.co/GpCKwDrfky,2024-05-14 16:36:59+00:00,[pic.twitter.com/GpCKwDrfky],None,https://pbs.twimg.com/media/GNjadkgacAAB0Gy?format=jpg&name=small,"This image is describing a stock short sale of 500 shares of the stock ""PARA"" at a limit price of 13.24. It is not depicting an option play like a call or a put.","\$PARA Closed\n\nIn 13.11 (yesterday)\n\nOut 13.24\n\n+1%\n+\$65 profit\n\nJust trying to reduce long exposure heading into tomorrow. https://t.co/GpCKwDrfky TRANSCRIBED IMAGE DATA: This image is describing a stock short sale of 500 shares of the stock ""PARA"" at a limit price of 13.24. It is not depicting an option play like a call or a put.","[PARA] [Sell]\n\nThis message indicates the user has sold their position in the stock ""PARA"" at a price of $13.24, which is higher than the price they initially bought it for ($13.11), resulting in a 1% profit or \$65 on the transaction. The intention was to reduce long exposure going into the next trading session."
1,1790415982328263151,"\$INTC Closed\n\nIn 30.53 (yesterday)\n\nOut 30.91\n\n+1.3%\n+\$152 profit\n\nLooking for a market dip tomorrow. Then reenter at a lower price, hopefully. https://t.co/VUk3AXsCDY",2024-05-14 16:16:49+00:00,[pic.twitter.com/VUk3AXsCDY],None,https://pbs.twimg.com/media/GNjV1NsbwAAwYeB?format=jpg&name=small,"The image shows details of a stock sale for Intel Corporation (INTC). This is a straightforward stock trade and not an option play like a call or put. Here are the details:\n\n- Ticker: INTC\n- The action is a sell order.\n- Quantity sold: 400 shares in total.\n- Sale price: \$30.91 per share.\n \nThis image reflects a limit sell order executed at a price of \$30.91 per share, and it is not related to options trading such as calls or puts.","\$INTC Closed\n\nIn 30.53 (yesterday)\n\nOut 30.91\n\n+1.3%\n+\$152 profit\n\nLooking for a market dip tomorrow. Then reenter at a lower price, hopefully. https://t.co/VUk3AXsCDY TRANSCRIBED IMAGE DATA: The image shows details of a stock sale for Intel Corporation (INTC). This is a straightforward stock trade and not an option play like a call or put. Here are the details:\n\n- Ticker: INTC\n- The action is a sell order.\n- Quantity sold: 400 shares in total.\n- Sale price: \$30.91 per share.\n \nThis image reflects a limit sell order executed at a price of \$30.91 per share, and it is not related to options trading such as calls or puts.",[INTC] [Sell]
2,1790077422697693598,Question:\n\nSell 1 naked GME call here.\n\nWorth it?\n\nDangerous?\n\nOnly 1 shouldn’t be too bad?,2024-05-13 17:51:30+00:00,0,No image available,None,None,Question:\n\nSell 1 naked GME call here.\n\nWorth it?\n\nDangerous?\n\nOnly 1 shouldn’t be too bad? TRANSCRIBED IMAGE DATA: None,"[GME] [Sell]\n\nThe message is discussing selling a naked call option for GME (GameStop). One person is asking if it is worth it and how dangerous it might be, indicating that selling just one contract might not be too risky."
3,1790077076105498901,\$PARA Open\n\nSee how Sony plays the buyout process.\n\nSmall. https://t.co/CceV23s8gb,2024-05-13 17:50:07+00:00,[pic.twitter.com/CceV23s8gb],None,https://pbs.twimg.com/media/GNehnPvbMAAq-N7?format=jpg&name=small,"This image describes a stock purchase, not an option play. The details are as follows:\n\n- The stock ticker is ""PARA.""\n- The purchase involved buying 500 shares at a limit price of \$13.11.\n- The order was filled at a price of \$13.1099 per share.\n- The transaction took place on May 13, 2024, at 10:46 AM.\n\nThis indicates a straightforward stock purchase and not a call or put option.","\$PARA Open\n\nSee how Sony plays the buyout process.\n\nSmall. https://t.co/CceV23s8gb TRANSCRIBED IMAGE DATA: This image describes a stock purchase, not an option pl

In [46]:
cwd = os.getcwd()
file_path = os.path.join(cwd, 'tweet_buy_or_sell.xlsx')
ht_mini['created_at'] = ht_mini['created_at'].dt.tz_localize(None)
ht_mini = ht_mini.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
print(file_path)
ht_mini.to_excel(file_path, index=False)
display(ht_mini)

C:\Users\amoog\AppData\Local\Temp\ipykernel_1284\1632751813.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ht_mini = ht_mini.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)


C:\Users\amoog\OneDrive\Desktop\Project_X\tweet_buy_or_sell.xlsx


,id,text,created_at,image_url,Image,jpg_url,image_response,full_response,buy_or_sell
0,1790421057742205068,\\\\\\\\$PARA Closed\\\\n\\\\nIn 13.11 (yesterday)\\\\n\\\\nOut 13.24\\\\n\\\\n+1%\\\\n+\\\\\\\\$65 profit\\\\n\\\\nJust trying to reduce long exposure heading into tomorrow. https://t.co/GpCKwDrfky,2024-05-14 16:36:59,[pic.twitter.com/GpCKwDrfky],None,https://pbs.twimg.com/media/GNjadkgacAAB0Gy?format=jpg&name=small,"This image is describing a stock short sale of 500 shares of the stock ""PARA"" at a limit price of 13.24. It is not depicting an option play like a call or a put.","\\\\\\\\$PARA Closed\\\\n\\\\nIn 13.11 (yesterday)\\\\n\\\\nOut 13.24\\\\n\\\\n+1%\\\\n+\\\\\\\\$65 profit\\\\n\\\\nJust trying to reduce long exposure heading into tomorrow. https://t.co/GpCKwDrfky TRANSCRIBED IMAGE DATA: This image is describing a stock short sale of 500 shares of the stock ""PARA"" at a limit price of 13.24. It is not depicting an option play like a call or a put.","[PARA] [Sell]\\\\n\\\\nThis message indicates the user has sold their position in the stock ""PARA"" at a price of $13.24, which is higher than the price they initially bought it for ($13.11), resulting in a 1% profit or \\\\\\\\$65 on the transaction. The intention was to reduce long exposure going into the next trading session."
1,1790415982328263151,"\\\\\\\\$INTC Closed\\\\n\\\\nIn 30.53 (yesterday)\\\\n\\\\nOut 30.91\\\\n\\\\n+1.3%\\\\n+\\\\\\\\$152 profit\\\\n\\\\nLooking for a market dip tomorrow. Then reenter at a lower price, hopefully. https://t.co/VUk3AXsCDY",2024-05-14 16:16:49,[pic.twitter.com/VUk3AXsCDY],None,https://pbs.twimg.com/media/GNjV1NsbwAAwYeB?format=jpg&name=small,"The image shows details of a stock sale for Intel Corporation (INTC). This is a straightforward stock trade and not an option play like a call or put. Here are the details:\\\\n\\\\n- Ticker: INTC\\\\n- The action is a sell order.\\\\n- Quantity sold: 400 shares in total.\\\\n- Sale price: \\\\\\\\$30.91 per share.\\\\n \\\\nThis image reflects a limit sell order executed at a price of \\\\\\\\$30.91 per share, and it is not related to options trading such as calls or puts.","\\\\\\\\$INTC Closed\\\\n\\\\nIn 30.53 (yesterday)\\\\n\\\\nOut 30.91\\\\n\\\\n+1.3%\\\\n+\\\\\\\\$152 profit\\\\n\\\\nLooking for a market dip tomorrow. Then reenter at a lower price, hopefully. https://t.co/VUk3AXsCDY TRANSCRIBED IMAGE DATA: The image shows details of a stock sale for Intel Corporation (INTC). This is a straightforward stock trade and not an option play like a call or put. Here are the details:\\\\n\\\\n- Ticker: INTC\\\\n- The action is a sell order.\\\\n- Quantity sold: 400 shares in total.\\\\n- Sale price: \\\\\\\\$30.91 per share.\\\\n \\\\nThis image reflects a limit sell order executed at a price of \\\\\\\\$30.91 per share, and it is not related to options trading such as calls or puts.",[INTC] [Sell]
2,1790077422697693598,Question:\\\\n\\\\nSell 1 naked GME call here.\\\\n\\\\nWorth it?\\\\n\\\\nDangerous?\\\\n\\\\nOnly 1 shouldn\\\\u2019t be too bad?,2024-05-13 17:51:30,0,No image available,None,None,Question:\\\\n\\\\nSell 1 naked GME call here.\\\\n\\\\nWorth it?\\\\n\\\\nDangerous?\\\\n\\\\nOnly 1 shouldn\\\\u2019t be too bad? TRANSCRIBED IMAGE DATA: None,"[GME] [Sell]\\\\n\\\\nThe message is discussing selling a naked call option for GME (GameStop). One person is asking if it is worth it and how dangerous it might be, indicating that selling just one contract might not be too risky."
3,1790077076105498901,\\\\\\\\$PARA Open\\\\n\\\\nSee how Sony plays the buyout process.\\\\n\\\\nSmall. https://t.co/CceV23s8gb,2024-05-13 17:50:07,[pic.twitter.com/CceV23s8gb],None,https://pbs.twimg.com/media/GNehnPvbMAAq-N7?format=jpg&name=small,"This image describes a stock purchase, not an option play. The details are as follows:\\\\n\\\\n- The stock ticker is ""PARA.""\\\\n- The purchase involved buying 500 shares at a limit price of \\\\\\\\$13.11.\\\\n- The order was filled at a price of 